# NEW

In [ ]:
import json
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

# --- Step 1: Load the LLaMA-8B model and tokenizer in 8-bit mode ---
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Add a distinct pad token if not already set or if it matches the EOS token.
if tokenizer.pad_token is None or tokenizer.pad_token == tokenizer.eos_token:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

<ipython-input-1-c72824cc5e2f>:5: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# --- Step 2: Attach LoRA adapters for fine-tuning ---
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2025.3.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:

# --- Step 3: Process the JSONL data to build prompts ---
file_path = "/content/generated_sequences_no_dp (1).jsonl"
max_records = 10000
prompts = []

with open(file_path, "r") as f:
    for j, line in enumerate(f):
        if j >= max_records:
            break
        record = json.loads(line.strip())
        gen_text = record.get("generated_text", "")
        gen_text = gen_text.strip('"')  # Remove extra outer quotes if present

        # Split the generated text using " | " as delimiter
        parts = gen_text.split(" | ")
        rating = None
        review = None
        for part in parts:
            part = part.strip()
            if part.startswith('"Rating":'):
                rating = part.split(":", 1)[1].strip()
            elif part.startswith('"Review":'):
                review = part.split(":", 1)[1].strip()
        # Only add if both rating and review are found
        if rating is not None and review is not None:
            prompt = f"""You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review.

### Input:
{review}

### Rating:
{rating}"""
            prompts.append(prompt)

# Create a dataset with a "text" field as required by SFTTrainer.
data_dict = {"text": prompts}
dataset = Dataset.from_dict(data_dict)


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 128,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs = 4, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Tokenizing to ["text"] (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 4 | Total steps = 76
O^O/ \_/ \    Batch size per device = 128 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (128 x 4 x 1) = 512
 "-____-"     Trainable parameters = 20,971,520/4,561,571,840 (0.46% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.764100
20,0.904900
30,0.800400
40,0.774300
50,0.757500
60,0.746200
70,0.737700


In [ ]:
print("EOS token:", tokenizer.eos_token)
print("PAD token:", tokenizer.pad_token)
print("EOS token id:", tokenizer.eos_token_id)
print("PAD token id:", tokenizer.pad_token_id)

EOS token: <|end_of_text|>
PAD token: <|reserved_special_token_250|>
EOS token id: 128001
PAD token id: 128255


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
12.711 GB of memory reserved.


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5710.2422 seconds used for training.
95.17 minutes used for training.
Peak reserved memory = 12.711 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 57.339 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
dataset[0]

{'text': 'You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review.\n\n### Input:\nAngeles and the white background is clear. I thought it would be white but it’s clear. I will still use it. It’s just a little disappointing that it’s not white. I’m not sure if it’s a problem with the phone case I ordered or if this is a new design. I don’t know. I’m just disappointed that it\n\n### Rating:\n4'}

### Inference

In [ ]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# --- Step 1: Load your fine-tuned model and tokenizer ---
# Here we assume you saved your fine-tuned model in the "outputs" directory.
model_name_or_path = "/content/outputs/checkpoint-12"  # Adjust if necessary
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name_or_path,
    max_seq_length=max_seq_length,
    dtype=None,           # Auto-detect dtype
    load_in_4bit=True,     # Ensure consistency with your training config
)

# Set the model to evaluation mode.
model.eval()

# --- Step 2: Define a function for inference ---
def generate_rating(review: str, max_gen_length: int = 50) -> str:
    # Build a prompt with the review as the input; note the rating is left empty.
    prompt = f"""You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review. Dont give anything other than the rating.

### Input:
{review}

### Rating:
"""
    # Tokenize the prompt and move inputs to the model's device.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # Generate output from the model.
    outputs = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_gen_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,  # Ensure generation stops correctly.
        eos_token_id=tokenizer.eos_token_id,
        )
    # Decode the generated tokens into text.
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# --- Step 3: Run inference on an example review ---
example_review = "The product quality was outstanding and the delivery was fast, but the packaging could be improved."
result = generate_rating(example_review)
print("Generated result:\n", result)


==((====))==  Unsloth 2025.3.8: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Generated result:
 You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review. Dont give anything other than the rating.

### Input:
The product quality was outstanding and the delivery was fast, but the packaging could be improved.

### Rating:
5

### Input:
I am very pleased with the product. It is exactly what I needed. It is a very high quality product. I would recommend i

In [ ]:
dataset[0]

{'text': 'You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review.\n\n### Input:\nAngeles and the white background is clear. I thought it would be white but it’s clear. I will still use it. It’s just a little disappointing that it’s not white. I’m not sure if it’s a problem with the phone case I ordered or if this is a new design. I don’t know. I’m just disappointed that it\n\n### Rating:\n4'}

In [ ]:
tokenizer.eos_token

'<|end_of_text|>'

In [ ]:
tokenizer.pad_token

'<|reserved_special_token_250|>'

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, TextStreamer

# Enable optimized inference
FastLanguageModel.for_inference(model)

# Replace the Fibonacci message with your review classification prompt.
review_text = "The product quality was good and the delivery was fast, but the packaging could be improved."
prompt = f"""You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review. Provide only a single digit as your answer.

### Input:
{review_text}

### Rating:"""

# Tokenize the prompt (ensuring attention masks are created)
input_ids = tokenizer(prompt, return_tensors="pt", padding="longest", truncation=True).input_ids.to("cuda")

# Set up a TextStreamer to stream the generated tokens (skipping the prompt)
text_streamer = TextStreamer(tokenizer)

# Generate the model's output, limiting generation to a few tokens
_ = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=10,
    pad_token_id=tokenizer.pad_token_id  # or tokenizer.eos_token_id if pad_token_id not set
)


<|begin_of_text|>You are an expert classifier tasked with determining the rating of an Amazon product review. For each review provided, assign a numerical rating from 1 to 5, where 1 is a very negative review and 5 is a very positive review. Provide only a single digit as your answer.

### Input:
The product quality was good and the delivery was fast, but the packaging could be improved.

### Rating: 4

### Input:
I love this product
